In [1]:
input_file = "day07/input"
lines = map(str.strip, open(input_file, "r").readlines())

import collections
import itertools

Hand = collections.namedtuple("Hand", ["cards", "bid"])
hands = list(itertools.starmap(lambda h, b: Hand(h, int(b)), map(lambda s: s.split(' '), lines)))

In [2]:
card_order = list("AKQJT98765432")

import functools
import enum

class HandType(enum.Enum):
    FIVE_OF_KIND = 1
    FOUR_OF_KIND = 2
    FULL_HOUSE = 3
    THREE_OF_KIND = 4
    TWO_PAIR = 5
    ONE_PAIR = 6
    HIGH_CARD = 7

def get_hand_type(hand):
    sh = [(k, len(list(v))) for k, v in itertools.groupby(sorted(hand))]
    
    if any([x for x in sh if x[1] == 5]):
        return HandType.FIVE_OF_KIND
    if any([x for x in sh if x[1] == 4]):
        return HandType.FOUR_OF_KIND
    if any([x for x in sh if x[1] == 3]) and any([x for x in sh if x[1] == 2]):
        return HandType.FULL_HOUSE
    if any([x for x in sh if x[1] == 3]):
        return HandType.THREE_OF_KIND
    if len([x for x in sh if x[1] == 2]) == 2:
        return HandType.TWO_PAIR
    if any([x for x in sh if x[1] == 2]):
        return HandType.ONE_PAIR
    return HandType.HIGH_CARD

def cmp_hands(h1, h2):
    t1 = get_hand_type(h1.cards)
    t2 = get_hand_type(h2.cards)
    if t1 == t2:
        # turn cards into their value
        i = itertools.starmap(lambda c1, c2: (card_order.index(c1), card_order.index(c2)), zip(h1.cards, h2.cards))
        # compute diff of card values between hands
        i = itertools.starmap(lambda c1, c2: c2 - c1, i)
        # return first non-zero difference
        return [d for d in i if d != 0][0]
    else:
        return t2.value - t1.value


sh = sorted(hands, key=functools.cmp_to_key(cmp_hands))

ranked = zip(map(lambda h: h.bid, sh), range(1, len(sh) + 1))
part1 = sum(itertools.starmap(int.__mul__, ranked))
print(f"Part 1: {part1}")

Part 1: 253910319


In [3]:
fst = lambda x: x[0]
snd = lambda x: x[1]

def sub_jokers(hand):
    sh = [(k, len(list(v))) for k, v in itertools.groupby(sorted(hand)) if k != 'J']
    cardinalities = list(map(snd, sh))
    ma = max(cardinalities) if cardinalities else 1
    replacement = [x[0] for x in sh if x[1] == ma][0] if cardinalities else 'A'
    return hand.replace('J', replacement)
    
card_order2 = list("AKQT98765432J")
def cmp_hands2(h1, h2):
    t1 = get_hand_type(sub_jokers(h1.cards))
    t2 = get_hand_type(sub_jokers(h2.cards))
    if t1 == t2:
        # turn cards into their value
        i = itertools.starmap(lambda c1, c2: (card_order2.index(c1), card_order2.index(c2)), zip(h1.cards, h2.cards))
        # compute diff of card values between hands
        i = itertools.starmap(lambda c1, c2: c2 - c1, i)
        # return first non-zero difference
        return [d for d in i if d != 0][0]
    else:
        return t2.value - t1.value

sh2 = sorted(hands, key=functools.cmp_to_key(cmp_hands2))

ranked = zip(map(lambda h: h.bid, sh2), range(1, len(sh2) + 1))
part2 = sum(itertools.starmap(int.__mul__, ranked))
print(f"Part 2: {part2}")

Part 2: 254083736
